In [10]:
import numpy as np
from plind.plmodel import plmodel
from plind.contour_dict import equilateral_real

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
def Gaussexp(z, lamb):
    return 1j*lamb*np.dot(z,z)

def gradh(z, lamb):
    return -2*1j*lamb*np.conj(z)

In [23]:
# init model
lamb = 1
delta = 0.1
Nstep = 100
dt = 1e-3

contour = equilateral_real(50, (-1,1,-1,1))

plind = plmodel(contour, Gaussexp, gradh=gradh, expargs=[lamb])

TypeError: __init__() got an unexpected keyword argument 'gradh'

In [13]:
plind.descend(dt, Nstep, delta)

ValueError: shapes (2500,2) and (2500,2) not aligned: 2 (dim 1) != 2500 (dim 0)

In [14]:
gradh = plind.get_grad()

In [19]:
gradh([[1.,3.],[2.,2.]],1)

/home/djow/.local/lib/python3.7/site-packages/autograd/tracer.py:14: UserWarning: Output seems independent of input.
  warnings.warn("Output seems independent of input.")


array([[-0., -0.],
       [-0., -0.]])